In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import calendar 
from datetime import datetime

In [13]:
df = pd.read_csv("/home/jonathan/tesis/11avo_avance/Acc_limpio.csv",sep=',')
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
print(DataAcc.shape)
DataAcc.head()

(30947, 4)


,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo


# Se selecciona los distritos que pueden tener influencia en el trayecto de estudio. Hortaleza, El pardo, Chamartín, chamberí

# Se descartan los siguientes tipos de accidentes: Caída, otro, atropello animal, solo salida de la vía, despeñamiento. Y se conservan aquellos que pueden afectar considerablemente el tráfico.

In [3]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8) | (DataAcc['cod_distrito'] == 5) | (DataAcc['cod_distrito'] == 7))# | (DataAcc['cod_distrito'] == 7) | (DataAcc['cod_distrito'] == 1) | (DataAcc['cod_distrito'] == 4))]  
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro') #& (DataAcc['tipo_accidente'] != 'Vuelco')
                        & (DataAcc['tipo_accidente'] != 'Atropello a animal') & (DataAcc['tipo_accidente'] != 'Colisión frontal')
                         & (DataAcc['tipo_accidente'] != 'Solo salida de la vía') & (DataAcc['tipo_accidente'] != 'Despeñamiento'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

In [4]:
DataAcc = DataAcc.reset_index()

# Redondeamos la hora al valor más cercano en punto, ya que los datos de los desplazamientos están registrados en horas exactas.

In [5]:
def ModTime(ds):
    global valor
    global fila  
    global prueba
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute    
    dia = date.day
    mes = date.month    
    consulta = DataAcc.loc[fila]    
    distrito = consulta['distrito']        
    
    if (distrito == 'FUENCARRAL-EL PARDO'):
        try:
            if (hora < 23) & (minuto >= 0):
                date = date.replace(hour=hora+1,minute=0)                        
            elif (hora == 23) & (minuto >= 0):
                date = date.replace(day=dia+1,hour=0,minute=0)                            
        except:            
            date = date.replace(month=mes+1, day=1, hour=0, minute=0)  
            
    elif (distrito == 'HORTALEZA'):
        try:
            if (hora < 23) & (minuto == 0):
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 23) & (minuto <= 45): ############REVISAR
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(day=dia+1,hour=0,minute=0)    
            elif (hora < 22) & (minuto > 45):        
                date = date.replace(hour=hora+2,minute=0)         
            elif (hora == 22) & (minuto > 45):
                date = date.replace(day=dia+1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(day=dia+1,hour=1,minute=0)
        except:               
            if (hora == 23) & (minuto == 0):
                date = date.replace(month=mes+1,day=1,hour=0)            
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
            elif (hora == 22) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)              
                
    elif (distrito == 'CHAMARTÍN'):           
        try:
            if (hora < 23) & (minuto == 0):                
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):                
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 22) & (minuto != 0):               
                date = date.replace(hour=hora+2,minute=0)            
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=0,minute=0)                
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=1,minute=0)                            
        except:            
            if (hora == 23) & (minuto == 0):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)    
                
    elif (distrito == 'CHAMBERÍ'):           
        try:
            if (minuto == 0):                
                date = date.replace(hour=hora,minute=0)            
            elif (minuto <= 10 ):               
                date = date.replace(hour=hora,minute=0)                       
            elif (hora < 23) & (minuto > 10):                
                date = date.replace(hour=hora+1,minute=0)                     
            elif (hora == 23) & (minuto > 10):                
                date = date.replace(day=dia+1,hour=0,minute=0)            
        except:            
            if (hora == 23):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
    fila += 1
    
    return (date)
prueba=0
fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ModTime)

# Agregamos columna ds2 que indica el aumento de los desplazamientos que ocurre 1 hora después de un accidente.

In [6]:
def ds2(ds):
    date = pd.to_datetime(ds)
    año = date.year
    mes = date.month
    dia = date.day
    hora = date.hour    
    try:
        if (hora < 23):   # Revisar si esta bien esta condición
            date = date.replace(hour=hora+1)        
        elif (hora == 23):
            date = date.replace(day=dia+1,hour=0)
    except:        
        if (mes < 12):
            date = date.replace(month=mes+1, day=1, hour=0)         
        else:
            date = date.replace(year=año+1, month=1, day=1, hour=0)
            
    return (date)

DataAcc['ds2'] = DataAcc['ds1'].apply(ds2)

In [7]:
DataAcc = DataAcc.set_index('ds')
Prueba = DataAcc.loc['2020-07-06':'2020-07-06']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
ds,,,,,,,
2020-07-06 10:45:00,2020S009407,7.0,CHAMBERÍ,Atropello a persona,True,2020-07-06 11:00:00,2020-07-06 12:00:00
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 13:00:00,2020-07-06 14:00:00
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-06 13:00:00,2020-07-06 14:00:00
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-07-06 18:00:00,2020-07-06 19:00:00
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 21:00:00,2020-07-06 22:00:00
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True,2020-07-06 22:00:00,2020-07-06 23:00:00


In [8]:
DataAcc = DataAcc.set_index('ds1')
Acc_ds1 = DataAcc.loc[:,['Accidente']]
DataAcc = DataAcc.set_index('ds2')
Acc_ds2 = DataAcc.loc[:,['Accidente']]

Acc_ds1 = Acc_ds1.reset_index() 
Acc_ds1 = Acc_ds1.drop_duplicates(subset = ['ds1'])
Acc_ds1 = Acc_ds1.set_index('ds1') 

Acc_ds2 = Acc_ds2.reset_index() 
Acc_ds2 = Acc_ds2.drop_duplicates(subset = ['ds2'])
Acc_ds2 = Acc_ds2.set_index('ds2') 

In [9]:
Acc_ds1

,Accidente
ds1,
2020-01-01 02:00:00,True
2020-01-01 10:00:00,True
2020-01-01 13:00:00,True
2020-01-01 15:00:00,True
2020-01-01 17:00:00,True
...,...
2020-12-31 14:00:00,True
2020-12-31 16:00:00,True
2020-12-31 18:00:00,True


In [10]:
Acc_ds2

,Accidente
ds2,
2020-01-01 03:00:00,True
2020-01-01 11:00:00,True
2020-01-01 14:00:00,True
2020-01-01 16:00:00,True
2020-01-01 18:00:00,True
...,...
2020-12-31 15:00:00,True
2020-12-31 17:00:00,True
2020-12-31 19:00:00,True


In [11]:
# Acc_ds1.to_csv("Acc_ds1.csv")
# Acc_ds2.to_csv("Acc_ds2.csv")